# Obtención, Procesamiento y Consolidación de Datos Climáticos  
## Región del Biobío (2020–2024)

Este notebook implementa un **pipeline de adquisición y preprocesamiento de datos climáticos históricos** para la **Región del Biobío**, cubriendo el período **2020–2024**.

El objetivo principal es construir un **dataset climático estructurado y reproducible**, apto para su integración con datos de incendios forestales en el contexto de análisis y modelamiento mediante técnicas de **Machine Learning**.

---

## Fuente de los datos

Los datos climáticos utilizados en este notebook se obtienen desde la plataforma **Open-Meteo**, la cual proporciona información meteorológica histórica basada en:

- Reanálisis atmosféricos
- Modelos numéricos de predicción
- Integración de múltiples fuentes meteorológicas globales

Open-Meteo entrega datos de libre acceso mediante una **API pública**, lo que permite la descarga automatizada y reproducible de variables climáticas sin necesidad de credenciales privadas.

---

## Descripción técnica del proceso

El flujo de trabajo implementado en este notebook contempla las siguientes etapas:

### 1. Definición espacial del estudio
- Selección de comunas pertenecientes a la Región del Biobío
- Asignación de coordenadas geográficas representativas (latitud y longitud) por comuna
- Incorporación de información administrativa (provincia y región)

### 2. Descarga de datos climáticos históricos
- Consulta a la API de Open-Meteo para cada ubicación definida
- Obtención de series temporales históricas para el período 2020–2024
- Variables descargadas a nivel diario:
  - Temperatura media del aire a 2 metros
  - Humedad relativa media
  - Precipitación diaria acumulada
  - Velocidad media del viento a 10 metros
  - Dirección media del viento a 10 metros

### 3. Preprocesamiento y limpieza
- Conversión de fechas a formato estándar
- Verificación de valores faltantes
- Normalización de nombres de columnas
- Consolidación de registros por fecha y comuna

### 4. Estructuración del dataset final
- Integración de variables climáticas con metadatos espaciales
- Organización del dataset en formato tabular
- Preparación del archivo para análisis exploratorio y modelamiento

### 5. Exportación
- Generación del archivo final en formato CSV:
  - `Condiciones_Climáticas_Biobio_2020_2024.csv`
- Almacenamiento del archivo dentro del directorio `Datasets_base/Dataset_clima`

---

## Rol del dataset dentro del estudio

El dataset generado mediante este notebook cumple un rol fundamental en el estudio, ya que:

- Proporciona el **contexto climático** asociado a eventos de incendios forestales
- Permite la generación de **variables explicativas** para modelos predictivos
- Facilita la creación de **variables rezagadas (lags)** para análisis temporal
- Sirve como base para estudios de correlación clima–incendios

---

## Consideraciones metodológicas

- Los datos corresponden a **valores estimados y agregados**, no a mediciones directas de estaciones meteorológicas en terreno.
- La resolución espacial está determinada por la representación puntual de cada comuna.
- Se recomienda evaluar la sensibilidad de los modelos frente a estas aproximaciones espaciales.

---

## Contexto académico

Este notebook forma parte de la **Actividad ML2**, enfocada en el desarrollo de un pipeline de datos reproducible para el análisis de incendios forestales mediante técnicas de ciencia de datos y aprendizaje automático.



In [ ]:
import requests
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# =====================================================
# 0. MAPEO COMUNA → PROVINCIA + REGIÓN + COORDENADAS
# =====================================================

provincia_map = {
    # Provincia Concepción
    "Concepción": "Concepción", "Talcahuano": "Concepción", "Hualpén": "Concepción",
    "San Pedro de la Paz": "Concepción", "Chiguayante": "Concepción", "Penco": "Concepción",
    "Tomé": "Concepción", "Hualqui": "Concepción", "Florida": "Concepción",
    "Santa Juana": "Concepción", "Lota": "Concepción", "Coronel": "Concepción",

    # Provincia Arauco
    "Arauco": "Arauco", "Cañete": "Arauco", "Contulmo": "Arauco",
    "Curanilahue": "Arauco", "Lebu": "Arauco", "Los Álamos": "Arauco", "Tirúa": "Arauco",

    # Provincia Biobío
    "Los Ángeles": "Biobío", "Mulchén": "Biobío", "Nacimiento": "Biobío",
    "Negrete": "Biobío", "Quilaco": "Biobío", "Quilleco": "Biobío",
    "San Rosendo": "Biobío", "Santa Bárbara": "Biobío",
    "Tucapel": "Biobío", "Antuco": "Biobío", "Cabrero": "Biobío",
    "Laja": "Biobío", "Yumbel": "Biobío", "Alto Biobío": "Biobío"
}

REGION_NAME = "Biobío"

coords_manual = {
    "Alto Biobío": (-37.8592, -71.4488)
}

# =====================================================
# 1. FUNCIÓN PARA OBTENER COORDENADAS
# =====================================================
def get_coords(place):
    if place in coords_manual:
        return coords_manual[place]

    url = (
        "https://geocoding-api.open-meteo.com/v1/search?"
        f"name={place}&count=1&language=es&format=json"
    )
    r = requests.get(url).json()
    if "results" not in r:
        print(f"⚠️ No se encontraron coordenadas para {place}, se omite.")
        return None, None

    lat = r["results"][0]["latitude"]
    lon = r["results"][0]["longitude"]
    return lat, lon


# =====================================================
# 2. CLIMA HORARIO → DIARIO (SOLO 5 VARIABLES)
# =====================================================
def get_weather_daily_from_hourly(lat, lon):
    url = (
        "https://archive-api.open-meteo.com/v1/archive?"
        f"latitude={lat}&longitude={lon}"
        "&start_date=2020-01-01&end_date=2024-12-31"
        "&hourly=temperature_2m,relativehumidity_2m,precipitation,"
        "windspeed_10m,winddirection_10m"
        "&timezone=America/Santiago"
    )

    r = requests.get(url).json()
    if "hourly" not in r:
        return pd.DataFrame()

    df = pd.DataFrame(r["hourly"])
    if df.empty:
        return df

    df["date"] = pd.to_datetime(df["time"]).dt.date

    # Convertir a numérico
    numeric_cols = ["temperature_2m", "relativehumidity_2m", "precipitation",
                    "windspeed_10m", "winddirection_10m"]

    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Agregación diaria
    agg_dict = {
        "temperature_2m": "mean",
        "relativehumidity_2m": "mean",
        "windspeed_10m": "mean",
        "winddirection_10m": "mean",
        "precipitation": "sum"
    }

    daily = df.groupby("date")[numeric_cols].agg(agg_dict).reset_index()

    daily.rename(columns={
        "temperature_2m": "temperature_2m_mean",
        "relativehumidity_2m": "relativehumidity_2m_mean",
        "windspeed_10m": "windspeed_10m_mean",
        "winddirection_10m": "winddirection_10m_mean",
        "precipitation": "precipitation_sum"
    }, inplace=True)

    return daily


# =====================================================
# 3. COMUNAS DEL BIOBÍO
# =====================================================
comunas_biobio = list(provincia_map.keys())


# =====================================================
# 4. LOOP PRINCIPAL
# =====================================================
all_data = []

for comuna in comunas_biobio:
    print(f"\n📍 Procesando comuna: {comuna}")

    lat, lon = get_coords(comuna)
    if lat is None:
        continue

    df_weather = get_weather_daily_from_hourly(lat, lon)

    if df_weather.empty:
        continue

    df = df_weather.copy()

    df["comuna"] = comuna
    df["provincia"] = provincia_map[comuna]
    df["region"] = REGION_NAME
    df["latitud"] = lat
    df["longitud"] = lon

    all_data.append(df)

# =====================================================
# 5. DATASET FINAL
# =====================================================
df_all = pd.concat(all_data, ignore_index=True)
df_all.to_csv("Biobio_2020_2024_Variables5.csv", index=False, encoding="utf-8-sig")
print("Archivo guardado: Biobio_2020_2024_Variables5.csv")

# =====================================================
# 6. NORMALIZACIÓN
# =====================================================
df_norm = df_all.copy()
cols_to_norm = df_norm.select_dtypes(include=["float64", "int64"]).columns
cols_to_norm = [c for c in cols_to_norm if c not in ["latitud", "longitud"]]

scaler = MinMaxScaler()
df_norm[cols_to_norm] = scaler.fit_transform(df_norm[cols_to_norm])

df_norm.to_csv("Condiciones_Climáticas_Biobio_2020_2024.csv",
               index=False, encoding="utf-8-sig")

print("Archivo guardado: Condiciones_Climáticas_Biobio_2020_2024.csv")




📍 Procesando comuna: Concepción

📍 Procesando comuna: Talcahuano

📍 Procesando comuna: Hualpén

📍 Procesando comuna: San Pedro de la Paz

📍 Procesando comuna: Chiguayante

📍 Procesando comuna: Penco

📍 Procesando comuna: Tomé

📍 Procesando comuna: Hualqui

📍 Procesando comuna: Florida

📍 Procesando comuna: Santa Juana

📍 Procesando comuna: Lota

📍 Procesando comuna: Coronel

📍 Procesando comuna: Arauco

📍 Procesando comuna: Cañete

📍 Procesando comuna: Contulmo

📍 Procesando comuna: Curanilahue

📍 Procesando comuna: Lebu

📍 Procesando comuna: Los Álamos

📍 Procesando comuna: Tirúa

📍 Procesando comuna: Los Ángeles

📍 Procesando comuna: Mulchén

📍 Procesando comuna: Nacimiento

📍 Procesando comuna: Negrete

📍 Procesando comuna: Quilaco

📍 Procesando comuna: Quilleco

📍 Procesando comuna: San Rosendo

📍 Procesando comuna: Santa Bárbara

📍 Procesando comuna: Tucapel

📍 Procesando comuna: Antuco

📍 Procesando comuna: Cabrero

📍 Procesando comuna: Laja

📍 Procesando comuna: Yumbel

📍 Proce

# Diagrama de flujo del pipeline climático

Inicio
  │
  ▼
Definición del área de estudio
(Región del Biobío – comunas y coordenadas)
  │
  ▼
Configuración de parámetros temporales
(Periodo 2020–2024)
  │
  ▼
Consulta a API Open-Meteo
(descarga de variables climáticas históricas)
  │
  ▼
Recepción de series temporales
(datos diarios por ubicación)
  │
  ▼
Preprocesamiento de datos
- Formato de fechas
- Limpieza de valores
- Normalización de columnas
  │
  ▼
Integración espacial
- Región
- Provincia
- Comuna
- Latitud / Longitud
  │
  ▼
Consolidación del dataset
(unificación de todos los registros)
  │
  ▼
Exportación a CSV
Condiciones_Climáticas_Biobio_2020_2024.csv
  │
  ▼
Fin